In [20]:
import cv2
import os
import time
import glob
import numpy as np
from scipy import misc
import PIL.Image
import io
import array
import re
import pathlib
import random

print('running')

data_dir = pathlib.Path('E:/School/MasterThesis/Week14/Gifs/Facade/base/')
save_dir = 'E:/School/MasterThesis/Week14/Gifs/Facade/200515facades/'
image_count = len(list(data_dir.glob('*.png')))

print('running')
print(image_count)

for file in data_dir.glob('*.png'):
    file = str(file)
    file = file.replace('\\', '/')
    res = re.findall("(\w+).png", file)
    filename = res[0]
    
    # Set to 0 if you want random floor heights
    floorCount = 0
    
    # Change here to change span of floorcounts
    if floorCount == 0:
        floorCount = random.randint(1,9)
        print(floorCount)
    
    image = cv2.imread(file)
    font = cv2.FONT_HERSHEY_COMPLEX 

    # Read the image
    height, width, channels = image.shape

    # convert to hsv
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # mask
    lower = np.array([0,0,0])
    upper = np.array([5,5,5])
    mask = cv2.inRange(hsv, lower, upper)

    plan = image.copy()

    # Make the masked areas uniform
    plan[np.where(mask!=0)] = 255
    plan[np.where(mask==0)] = 0

    # Treshold the image and find contours
    _, plan_tresh = cv2.threshold(cv2.cvtColor(plan, cv2.COLOR_BGR2GRAY), 45, 255, cv2.THRESH_BINARY)
    plan_tresh =~ plan_tresh
    cnts, heirarchy = cv2.findContours(plan_tresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    c = max(cnts, key=cv2.contourArea) 

    approx = cv2.approxPolyDP(c, 1, True) 

    # draws boundary of contours. 
    #cv2.drawContours(image, [approx], 0, (255, 0, 255), 3)  

    # calculate moments of binary image
    M = cv2.moments(plan_tresh)

    # calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    # Used to flatted the array containing 
    # the co-ordinates of the vertices. 
    n = approx.ravel()  
    tuples = []

    # create tuples of the list
    for i,k in zip(n[0::2], n[1::2]):
        tuples.append([i, k])

    i = 0
    topy = 0
    topx = 0
    lowy = width
    lowx = width
    topRight = width
    topLeft = width
    botRight = 0
    botLeft = 0
    leftTop = width
    leftBot = width
    rightTop = 0
    rightBot = 0


    for j in tuples:
        x = j[0]
        y = j[1]

        if(topy < y):
            topy = y
        if(lowy > y):
            lowy = y
        if(topx < x):
            topx = x
        if(lowx > x):
            lowx = x

    for j in tuples:
        x = j[0]
        y = j[1]

        # finding points for top facade
        if(lowx == x):
            if(topLeft > y):
                topLeft = y
        if(topx == x):
            if(topRight > y):
                topRight = y

        # finding points for bot facade
        if(lowx == x):
            if(botLeft < y):
                botLeft = y
        if(topx == x):
            if(botRight < y):
                botRight = y        

        # finding points for left facade
        if(lowy == y):
            if(leftTop > x):
                leftTop = x
        if(topy == y):
            if(leftBot > x):
                leftBot = x   

        # finding points for right facade
        if(lowy == y):
            if(rightTop < x):
                rightTop = x
        if(topy == y):
            if(rightBot < x):
                rightBot = x   



    # paring the values in tuples 
    topLeftT = tuple((lowx, topLeft))
    topRightT = tuple((topx, topRight))
    botLeftT = tuple((lowx, botLeft))
    botRightT = tuple((topx, botRight))
    leftTopT = tuple((leftTop, lowy))
    leftBotT = tuple((leftBot, topy))
    rightTopT = tuple((rightTop, lowy))
    rightBotT = tuple((rightBot, topy))

    # this defines how much the indent of the facade mask is so we only get facade windows.
    indent = 5

    # a mask is created above each of the broken out parts so that we can select it
    # mask for top facade
    topMask = np.zeros((width,height), np.uint8)
    topPts = np.array([[0, 0], [width, 0], topRightT, 
                       [topx, topRight+indent], [lowx, topLeft+indent], topLeftT])
    _=cv2.drawContours(topMask, np.int32([topPts]),0, 255, -1)

    # mask for bot facade
    botMask = np.zeros((width,height), np.uint8)
    botPts = np.array([[0, height], [width, height], botRightT, 
                       [topx, botRight-indent], [lowx, botLeft-indent], botLeftT])
    _=cv2.drawContours(botMask, np.int32([botPts]),0, 255, -1)

    # mask for left facade
    leftMask = np.zeros((width,height), np.uint8)
    leftPts = np.array([[0, 0], [0, height], leftBotT, 
                        [leftBot+indent, topy], [leftTop+indent, lowy], leftTopT])
    _=cv2.drawContours(leftMask, np.int32([leftPts]),0, 255, -1)

    # mask for right facade
    rightMask = np.zeros((width,height), np.uint8)
    rightPts = np.array([[width, height], [width, 0], rightTopT, 
                         [rightTop-indent, lowy], [rightBot-indent, topy], rightBotT])
    _=cv2.drawContours(rightMask, np.int32([rightPts]),0, 255, -1)

    # everything except the mask is now erased from the image
    topImg = image.copy()
    topImg[topMask==0] = 0

    botImg = image.copy()
    botImg[botMask==0] = 0

    leftImg = image.copy()
    leftImg[leftMask==0] = 0

    rightImg = image.copy()
    rightImg[rightMask==0] = 0

    # Rotate images so all facades are facing up
    # Top Image is fine so we ignore it
    botImg = cv2.rotate(botImg, cv2.ROTATE_180)
    leftImg = cv2.rotate(leftImg, cv2.ROTATE_90_CLOCKWISE)
    rightImg = cv2.rotate(rightImg, cv2.ROTATE_90_COUNTERCLOCKWISE)

    # This gets us our starting points for the facade
    def startingPoints(botTuple, topTuple, angle):
        mask = np.zeros((width,height), np.uint8)
        cv2.line(mask, botTuple, topTuple, (255, 255, 255), 1) 

        m = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)
        dst = cv2.warpAffine(mask, m, (width, height))

        cnt, heirarchy = cv2.findContours(dst, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        _=cv2.drawContours(dst, cnt, 0, (255, 0, 255), 3)

        botx = 5000
        topx = 0 

        #find highest and lowest x values
        for c in cnt[0]:
            for x,y in c:
                if(botx > x):
                    botx = x
                if(topx < x):
                    topx = x
        return botx, topx

    # This function contours the windows in each plan, might want to add balconies aswell
    def contourW(img):
        # convert to hsv
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        # mask the windows so they can be countoured and counted
        lower = np.array([0,0,120])
        upper = np.array([10,10,150])
        mask = cv2.inRange(hsv, lower, upper)

        cnt, heirarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        return cnt

    # This function return the x cordinates of the windows on the facade
    def windowPoints(cnt):
        # make it a numpy array
        arr = np.asarray(cnt)

        # get how many windows there is on the facade
        res = re.findall("(\w+),", str(arr.shape))
        loops = int(res[0])

        botx = 5000
        topx = 0
        boty = 5000
        topy = 0
        i = 0
        windowArr = []

        # loop to find the x values of each window
        while i < loops:
            #find highest and lowest x values
            for c in cnt[i]:
                for x,y in c:
                    if(botx > x):
                        botx = x
                    if(topx < x):
                        topx = x
                    if(boty > y):
                        boty = y
                    if(topy < y):
                        topy = y
            if((topx-botx)<=(topy-boty)):
                None
            else:        
                windowArr.append(botx)
                windowArr.append(topx)

            # reset variable between adding each window
            botx = 5000
            topx = 0
            boty = 5000
            topy = 0
            i += 1
        return windowArr 

    # This draws the windows on the specified facade
    def drawWindows(arg, facade):
        window2Floor = (height/(floorCount+1))/4
        window2Roof = window2Floor*3
        floorHeight = height/(floorCount+1)
        iL = 0
        while iL < floorCount:
            i = 0
            n = 0
            arr = []

            for point in arg:
                if i % 2 == 0:
                    arr.append([])
                    arr[n].insert(0, point)
                    arr[n].insert(1, int((height-window2Floor) - (floorHeight*iL)))
                else:
                    arr[n].insert(0, point)
                    arr[n].insert(1, int((height-window2Roof) - (floorHeight*iL)))
                    n += 1
                i += 1

            for point in arr:
                # Remember this is in BGR not RGB 
                cv2.rectangle(facade, (point[0], point[1]), (point[2], point[3]), (255, 85, 0), -1)

            iL += 1
        # print('Drew ' + str(len(arg)/2) + ' windows in ' + str(floorCount) + ' rows on facade')
        return facade

    # This draws facades on specified side
    def generateFacade(botT, topT, img, side, angle):
        # print('\nGenerating ' + side + ' facade')
        # starting points of facade
        x1, x2 = startingPoints(botT, topT, angle)

        #print('x1 = ' + str(x1) + ' x2 = ' + str(x2))

        # could bake this into the windowPoints function
        windowCnt = contourW(img)

        # this could be inside countourW, but here we get x cordinates for the windows
        wArr = windowPoints(windowCnt)
        
        # Remember this is in BGR not RGB 
        facade = np.full((width, height, 3), (222, 0, 0))

        # Draw building volume using starting points
        cv2.rectangle(facade, (x1, height), (x2, (height-int((height/(floorCount+0.5))*floorCount))), (255, 48, 0), -1)

        # Here we draw the windows that we got from the points
        leftFacade = drawWindows(wArr, facade)

        is_success, genFacade = cv2.imencode('.png', facade)
        
        # This is to only save facades with windows on, removing for GIF generation
        '''
        if len(wArr) == 0:
            print('Array is empty')
            print('Not saving ' + side + ' facade')
        else:
            print('Array has ' + str(int(len(wArr)/2)) + ' windows in it')
            print('Saving facade as ' + side + '_facade.png')

            buf = io.BytesIO(genFacade)
            buf.seek(0)

            im_plan = PIL.Image.open(buf)
            im_plan.save(save_dir + filename + '_' + side + '.png')

            buf.close()
        '''
        buf = io.BytesIO(genFacade)
        buf.seek(0)

        im_plan = PIL.Image.open(buf)
        im_plan.save(save_dir + filename + '_' + side + '.png')

        buf.close()
        
    generateFacade(leftBotT, leftTopT, leftImg, 'left', 90)
    generateFacade(rightBotT, rightTopT, rightImg, 'right', 270)
    generateFacade(topLeftT, topRightT, topImg, 'top', 0)
    generateFacade(botRightT, botLeftT, botImg, 'bot', 180)
    
    # Debug to show where cuts in plan drawing are, removing for GIFS
    is_success, rightImg = cv2.imencode('.png', rightImg)
    is_success, leftImg = cv2.imencode('.png', leftImg)
    is_success, topImg = cv2.imencode('.png', topImg)
    is_success, botImg = cv2.imencode('.png', botImg)
    
    buf = io.BytesIO(rightImg)
    buf.seek(0)

    im_plan = PIL.Image.open(buf)
    im_plan.save('E:/Tests/1/'+ str(filename) +'rightImg.png')

    buf.close()

    buf = io.BytesIO(leftImg)
    buf.seek(0)

    im_plan = PIL.Image.open(buf)
    im_plan.save('E:/Tests/1/'+ str(filename)  +'leftImg.png')

    buf.close()

    buf = io.BytesIO(topImg)
    buf.seek(0)

    im_plan = PIL.Image.open(buf)
    im_plan.save('E:/Tests/1/'+ str(filename)  +'topImg.png')

    buf.close()

    buf = io.BytesIO(botImg)
    buf.seek(0)

    im_plan = PIL.Image.open(buf)
    im_plan.save('E:/Tests/1/'+ str(filename)  +'botImg.png')

    buf.close()
print('\nDone')

running
running
600
7
3
5
2
9
1
7
2
7
6
4
6
6
4
2
6
1
4
7
1
8
5
9
4
1
5
4
1
6
4
3
9
3
5
9
2
3
8
9
7
6
5
5
3
1
4
6
1
8
5
4
9
3
8
5
8
5
5
7
9
3
4
9
5
5
4
9
9
4
2
9
5
4
3
5
5
4
2
5
1
2
5
1
3
4
9
1
7
5
9
4
1
1
2
9
9
7
5
8
2
1
9
1
7
7
4
2
6
2
9
7
4
4
7
2
8
8
9
4
8
3
9
2
3
5
9
1
6
6
6
2
9
8
1
1
5
4
2
6
1
6
8
4
5
1
7
8
9
1
5
1
9
8
5
5
6
2
6
2
5
7
6
9
8
4
4
5
6
1
7
9
7
5
1
7
2
5
3
1
8
1
1
3
2
6
6
3
5
2
1
7
1
4
5
1
6
4
8
3
4
8
1
4
2
2
1
9
2
2
5
5
6
6
6
2
1
4
6
5
9
1
8
8
2
2
2
6
8
2
5
2
1
3
6
9
3
7
3
1
1
2
9
3
8
3
4
3
9
6
3
1
8
7
3
1
3
1
3
9
8
8
6
5
3
4
8
7
5
9
5
5
7
8
1
9
7
9
8
3
7
1
2
2
8
8
5
6
7
9
3
3
6
4
4
4
4
2
8
1
5
9
1
8
1
9
9
9
1
1
3
7
9
3
2
2
5
3
4
7
6
3
1
1
9
6
5
7
6
3
1
4
1
1
4
2
3
8
3
5
1
3
3
5
3
7
1
4
5
3
5
3
5
1
8
4
3
1
7
3
6
4
2
5
2
4
5
4
9
2
4
2
7
5
8
2
7
5
6
9
9
4
1
7
6
9
1
2
4
2
3
9
8
8
9
5
7
4
4
9
2
1
3
1
1
4
4
4
5
4
6
7
2
3
1
3
1
1
1
6
3
5
5
6
6
1
6
5
6
6
7
3
2
8
6
4
9
6
1
6
8
9
6
5
6
2
6
9
9
5
4
7
5
8
7
3
8
9
6
7
1
6
7
5
5
6
9
9
6
9
7
4
1
5
8
4
3
5
6
8
2
4
4
1
2
2
5
9
4
8
8
